In [11]:
!pip install pyvoronoi

  Using cached pyvoronoi-1.0.7.tar.gz (147 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [14 lines of output]
      Distribution mode: Compiling Cython generated .cpp sources.
      running bdist_wheel
      running build
      running build_ext
      building 'pyvoronoi' extension
      creating build
      creating build/temp.linux-aarch64-cpython-311
      creating build/temp.linux-aarch64-cpython-311/pyvoronoi
      gcc -pthread -B /opt/conda/compiler_compat -Wsign-compare -DNDEBUG -fwrapv -O3 -Wall -fPIC -O3 -isystem /opt/conda/include -fPIC -O3 -isystem /opt/conda/include -fPIC -Ipyvoronoi -I/opt/conda/include/python3.11 -c pyvoronoi/pyvoronoi.cpp -o build/temp.linux-aarch64-cpython-311/pyvoronoi/pyvoronoi.o
      pyvoronoi/pyvoronoi.cpp:198:12: fatal error: longintrepr.h: No such file or directory
        198 |   #include "longintrepr.h"
            |         

In [7]:
file = ('../../drive/energy/renewables/docs/uittreksels/20180627-1031_66937957_NL.pdf')

In [10]:
from py_pdf_parser.loaders import load_file
from py_pdf_parser.visualise import visualise

ModuleNotFoundError: No module named 'pyvoronoi'

In [ ]:
doc = load_file(file)

In [ ]:
for element in doc.elements:
    print(element.text())

In [ ]:
PATH_DATA = '../../drive/energy/renewables/'

In [ ]:
reader = PdfReader('../../drive/energy/renewables/docs/uittreksels/20180627-1031_66937957_NL.pdf')

In [ ]:
pagenumber = 0
text = ''
    
while pagenumber < len(reader.pages):
    page = reader.pages[pagenumber]
    text += page.extract_text()
    pagenumber += 1

In [ ]:
print(text)

In [ ]:
with open('../../config/companyinfo.yaml', 'r') as file:
    patterns = yaml.safe_load(file)

In [ ]:

def get_bestuurders(pdf_file):

    # Parse pdf to text

    reader = PdfReader(pdf_file)
    
    pagenumber = 0
    text = ''
    
    while pagenumber < len(reader.pages):
        page = reader.pages[pagenumber]
        text += page.extract_text()
        pagenumber += 1

    # Get regex patterns for uittreksels (import from YAML

    uit = patterns.get('uittreksel')

    # Compile regex
    kvk_nummer = int(re.findall(re.compile(uit.get('kvk_nummer', 'Unknown')), text)[0].strip())
    rsin = int(re.findall(re.compile(uit.get('rsin', 'Unknown')), text)[0])
    rechtsvorm = re.findall(re.compile(uit.get('rechtsvorm', 'Unknown')), text)[0].strip()
    stat_naam = re.findall(re.compile(uit.get('stat_naam', 'Unknown')), text)[0].strip()
    stat_zetel = re.findall(re.compile(uit.get('stat_zetel', 'Unknown')), text)[0].strip()
    inschrijving = re.findall(re.compile(uit.get('inschrijving', 'Unknown')), text)[0].strip()
    handelsnaam = re.findall(re.compile(uit.get('handelsnaam', 'Unknown')), text)[0].strip()
    sbi_codes = list(set(re.findall(re.compile(uit.get('sbi_codes', 'Unknown')), text)))
    vestigingsnr = re.findall(re.compile(uit.get('vestigingsnr', 'Unknown')), text)[0] # leave as srtring because of padded zeroes
    b_adres = re.findall(re.compile(uit.get('b_adres', 'Unknown')), text)[0].strip()
    #p_adres = re.findall(re.compile(uit.get('p_adres', 'Unknown')), text)[0].strip()
    tel = re.findall(re.compile(uit.get('tel', 'Unknown')), text)[0].strip() # leave as string because of padded zero
    bestuurders = re.findall(re.compile(uit.get('bestuurders'), re.DOTALL), text)[0].split('Naam')
    #best_naam = re.findall(re.compile(uit.get('best_naam', 'Unknown')), text)
    #best_gebdat = re.findall(re.compile(uit.get('best_gebdat', 'Unknown')), text)
    #best_in_functie = re.findall(re.compile(uit.get('best_in_functie', 'Unknown')), text)
    #best_titel = re.findall(re.compile(uit.get('best_titel', 'Unknown')), text)[0]
    #best_bevoegd = re.findall(re.compile(uit.get('best_bevoegd', 'Unknown')), text)
    dat_uittreksel = re.findall(re.compile(uit.get('dat_uittreksel', 'Unknown')), text)[0]

    # Parse files

    records = [] # Create empty list

    # Iterate through text
    
    for bestuurder in bestuurders:
        
        record = {'kvk_nummer': kvk_nummer,
                   'rsin': rsin,
                   'rechtsvorm': rechtsvorm,
                   'statutaire_naam': stat_naam,
                   'statutaire_zetel': stat_zetel,
                   'inschrijfdatum': inschrijving,
                   'handelsnaam': handelsnaam,
                   'sbi_codes': sbi_codes,
                   'vestigingsnummer': vestigingsnr,
                   'bezoekadres': b_adres,
                   #'postadres': p_adres,
                   'telefoon': tel,
                   'datum_uittreksel': dat_uittreksel}
        
        record['naam_bestuurder'] = re.findall('(?<= )(.*)(?=\nGeboortedatum)', bestuurder)[0]
        record['geboortedatum'] = re.findall('\d{2}-\d{2}-\d{4}', bestuurder)[0]
        record['bestuurder_in_functie'] = re.findall('\d{2}-\d{2}-\d{4}', bestuurder)[1]
        record['titel_bestuurder'] = re.findall('(?<=Titel )(.*)(?=\nBevoegdheid)', bestuurder)[0]        
        
        records.append(record)

        # Create dataframe

    df = pd.DataFrame(records)

    return df

In [ ]:
bestuurders

In [ ]:
from glob import glob
dfs = []
for file in glob(PATH_DATA + 'docs/uittreksels/*.pdf'):
    print(file)
    df = get_bestuurders(file)
    dfs.append(df)

In [ ]:
df = pd.concat(dfs)
df.datum_uittreksel = pd.to_datetime(df.datum_uittreksel, dayfirst=True)
df = df.sort_values(by='datum_uittreksel', ascending=False)
df.drop_duplicates(subset=['naam_bestuurder',  'kvk_nummer', 'bestuurder_in_functie'], inplace=True)

In [ ]:
df

In [ ]:
records = []

for naam, datum, in_functie, titel, bevoegd in zip(best_naam, best_gebdat, best_in_functie, best_titel, best_bevoegd):
    record = {'kvk_nummer': kvk_nummer,
           'rsin': rsin,
           'rechtsvorm': rechtsvorm,
           'statutaire_naam': stat_naam,
           'statutaire_stad': stat_city,
           'statutair_land': stat_country,
           'inschrijfdatum': inschrijving,
           'handelsnaam': handelsnaam,
           'sbi_codes': sbi_codes,
           'vestigingsnummer': vestigingsnr,
           'bezoekadres': b_adres,
           'postadres': p_adres,
           'telefoon': tel,
           'datum_uittreksel': dat_uittreksel}
    
    record['naam_bestuurder'] = naam
    if len(best_gebdat) == len(best_naam):
        record['geboortedatum'] = datum.split(', ')[0]
        record['geboorteplaats'] = datum.split(', ')[1]
    if len(best_in_functie) == len(best_naam):
        record['bestuurder_in_functie'] = in_functie
    if len(best_titel) == len(best_naam):
        record['titel_bestuurder'] = titel
    if len(best_bevoegd) == len(best_naam):
        record['bevoegdheid_bestuurder'] = bevoegd
    records.append(record)

In [ ]:
pd.DataFrame(records)

In [ ]:
best_gebdat = re.findall(uit.get('best_gebdat'), text)
best_stad = re.findall(uit.get('best_gebdat'), text)[1].split(', ')[1]
best_country = re.findall(uit.get('best_gebdat'), text)[1].split(', ')[2]

In [ ]:
best_gebdat

In [ ]:
patterns = ['kvk_nummer', 'rsin', '

In [ ]:

rechtsvorm = re.findall(re.compile(uit.get('rechtsvorm'), re.DOTALL), text)[0]
stat_naam = re.findall(re.compile(uit.get('stat_naam'), re.DOTALL), text)[0]
stat_zetel = re.findall(re.compile(uit.get('stat_zetel'), re.DOTALL), text)[0]
inschrijving = re.findall(re.compile(uit.get('inschrijving'), re.DOTALL), text)[0]
handelsnaam  = re.findall(handelsnaam, text)[0]
sbi_codes = re.findall(sbi_codes, text)
vestigingsnr = re.findall(vestigingsnr, text)[0] # leave as string because of padded zeroes


In [ ]:
# Parse uittreksels

kvk_nummer = re.compile('(?<=KvK-nummer  )(.*)(?=\\n)')
rechtspersoon = re.compile('(?<=Rechtspersoon\\nRSIN\s)([0-9]+)\\n')
rechtsvorm = re.compile('Rechtsvorm ([A-Za-z ]+)\\n')
stat_naam = re.compile('Statutaire naam ([A-Za-z \u0080-\uFFFF\.0-9]+)\\n')
stat_zetel = re.compile('Statutaire zetel ([A-Za-z, \u0080-\uFFFF]+)\\n')
inschrijving = re.compile('Eerste inschrijving handelsregister (\d{2}-\d{2}-\d{4})\\n')
handelsnaam = re.compile('Handelsnaam ([A-Za-z \u0080-\uFFFF\.0-9]+)\\n')
sbi_codes = re.compile('SBI-code: ([0-9 \-A-Za-z]+)\\n')
vestigingsnr = re.compile('Vestigingsnummer ([0-9]+)\\n')
b_adres = re.compile('\nBezoekadres ([A-Za-z0-9 \u0080-\uFFFF.,]+)\\n')
p_adres = re.compile('Postadres ([A-Za-z0-9 \u0080-\uFFFF.,]+)\\n')
tel = re.compile('Telefoonnummer ([0-9]+)\\n')
best_naam = re.compile('\nNaam ([A-Za-z \u0080-\uFFFF,\.\-]+)\\n')
best_gebdat = re.compile("Geboortedatum en -plaats ([0-9A-Za-z \-\u0080-\uFFFF,\(\)\'\.]+)\\n")
best_in_functie = re.compile('Datum in functie (\d{2}-\d{2}-\d{4}) ')
best_titel = re.compile('Titel ([A-Za-z]+)\\n')
best_bevoegd = re.compile('Bevoegdheid ([A-Za-z \(\),\.]+)\\n')
dat_uittreksel = re.compile('Uittreksel is vervaardigd op (\d{2}-\d{2}-\d{4})')

In [ ]:
text